In [ ]:
%matplotlib inline
import numpy as np 
import nglview as nv
import pytraj as pt
from falass import readwrite, job, sld, reflect, compare

# ISIS Neutron Training Course
## MD-based analysis of neutron reflectometry

#### Andrew R. McCluskey 
##### University of Bath/Diamond Light Source - arm61@bath.ac.uk

2018-03-08

In this exercise, you will learn to analyse a neutron reflectometry dataset using a molecular dynamics (MD) simulation. The aim of this exercise is to understand how reflectometry experiments can be analysed using MD, this notebook details how falass [1] can be used to determine neutron reflectometry from a simulation trajectory. API documentation for falass is available at https://readthedocs.org/projects/falass/.

The first thing to do with falass is to read in some experimental data, there is an example data set and simulation in the `example` directory. This can be read using the following python command:

```
readwrite.Files(datfile='filepath')
```

So to read the example dataset we run the following. 

In [ ]:
files = readwrite.Files(datfile='example/example.dat')

These .dat files are three column space separated files consisting of information about q, reflected intensity and an uncertainty in reflected intensity.

In [ ]:
cat example/example.dat

The above command simply defines the datfile in the files class, these now must be read into computer memory.

In [ ]:
files.read_dat()

We can also plot this data to check that the right file has been read in. 

In [ ]:
expdata = files.plot_dat(rq4=False)
expdata.show()

With the experimental data, it is then necessary to get an associated simulation. Since we are simulated a monolayer, it is best to create a monolayer-like starting structure. This has been done for you, using the software packmol and the 'make_mono.inp' script, this will be demonstrated by the lecturer. This results in a monolayer of very low surface coverage.

In [ ]:
mono_init = pt.load('example/monolayer.pdb')
view = nv.show_pytraj(mono_init)
view

Now it is necessary to solvate the structure -- technically this could be achieved with packmol, however in order to get the structure shown below a more involved method using GROMACS utilities was used (if you are intereseted speak to the lecturer). You can view this structure using VMD (using nglview will likely crash the computer due to the size). 

This is a suitable starting structure for our simulations. Now we must find an accurate force-field, luckly as we are studying a lipid system many force-fields exist (as the study of lipids are important to the rationalisation of membrane proteins, amoung other systems). A great resource for lipid force-fields is the [lipidbook](https://lipidbook.bioch.ox.ac.uk/), this is a public repository for force-field parameters for lipids that is curated by the lipid biophysics group at Oxford University. Visit this website and try and find a suitable set of parameters, note we will be performing the simulations using GROMACS. 

You will quickly note that no parameters exist on this site for DSPC, however there are parameters for DPPC. This lipid is very similar to DSPC (just has two fewer carbons in each chain). This means that we can do a common computationalist practice of edited the force-field parameters to apply it to our model. Download the DPPC parameters and consider how they could be edited to be applied to DSPC. To save you effort or actually changing the force-field files (a non-trivial task) we have included the modified files (dspc.top), try comparing the two sets of parameters. 

Now that we have a starting structure and a set of force-field parameters it is possible to perform the simulation. In the data directory, you will find two .mdp files; namely squeeze.mdp and equil.mdp. The former allows for the performance of an in-silico Langmuir trough, where the pressure is greater in the x- and y-dimensions than it is the in z-dimension. 

The way that simulations are set up is dependent on the program used. For GROMACS the syntax is:

```
gmx grompp -f settings.mdp -o output.tpr -c structure.pdb -p force.top
```

This will produce a .tpr file that for which the simulation can be run using, 

```
gmx mdrun -deffnm -v output
```

Since these simulations take a long time to run. There are a pair of completed simulations in the data file, named low_apm.pdb and high_apm.pdb. These can be viewed with VMD. 

It is now necessary to read this simulation file into falass.

In [ ]:
files.pdbfile = 'example/example.pdb'
files.flip = True
files.read_pdb()

With the trajectory read in we must give scattering lengths to each of the different atom types. This is achieved through a .lgt file. 

In [ ]:
cat example/example.lgt

In [ ]:
files.lgtfile = 'example/example.lgt'
files.read_lgt()

These three files are brought together in the Job object and the scattering lengths are assigned to the different atom types. 

In [ ]:
layer_thickness = 1.
cut_off = 5.

job = job.Job(files, layer_thickness, cut_off)
job.set_lgts()

It is then possible to determine the scattering length denisty profiles as follows. 

In [ ]:
sld = sld.SLD(job)
sld.get_sld_profile()
sld.average_sld_profile()
sld.plot_sld_profile()

This scattering length density profile is then converted to a reflectometry profile as follows.

In [ ]:
reflect = reflect.Reflect(sld.sld_profile, files.expdata)
reflect.calc_ref()
reflect.average_ref() 
reflect.plot_ref(rq4=False)

Finally the experimental data can be compared with the data arising from simulation. 

In [ ]:
compare = compare.Compare(files.expdata, reflect.averagereflect, 1e-1, 1e-6)
compare.fit()
compare.return_fitted()
compare.plot_compare()

Now open a new notebook and analyse the high_apm.pdb and low_apm.pdb trajectories for each of the following contrasts:

- d$_{83}$-DSPC in ACMW
- d$_{70}$-DSPC in ACMW
- d$_{83}$-DSPC in D$_2$O
- d$_{70}$-DSPC in D$_2$O
- h-DSPC in D$_2$O

All of the datasets, the simulation trajectories and some of the lgtfiles are found in the `data` directory. Determine which trajectory better represents the experimental system under study. Once you have found the correct simulation discuss how the use of simulation-based analysis methods could improve the data obtained from reflectometry experiments. 

[1] *falass*, Andrew R. McCluskey, (http://people.bath.ac.uk/arm61/falass/)